In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


In [2]:
# Generate synthetic data
np.random.seed(42)
data_normal = np.random.normal(0, 1, (1000, 2))  # Normal data
data_anomalous = np.random.normal(0, 5, (50, 2))  # Anomalous data
data = np.vstack([data_normal, data_anomalous])

# Create a DataFrame
df = pd.DataFrame(data, columns=['feature1', 'feature2'])
df['label'] = [0] * 1000 + [1] * 50  # 0: Normal, 1: Anomalous


In [3]:
# Separate features and labels
X = df.drop(columns='label')
y = df['label']

# Imputation (if needed)
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)

# Standardization
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)


In [4]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Train Isolation Forest model
model = IsolationForest(contamination=0.05, random_state=42)
model.fit(X_train)

# Predict anomalies
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

# Convert predictions from {-1, 1} to {0, 1}
y_pred_train = np.where(y_pred_train == -1, 1, 0)
y_pred_test = np.where(y_pred_test == -1, 1, 0)

# Evaluate the model
print("Training Set Evaluation:")
print(classification_report(y_train, y_pred_train))

print("Test Set Evaluation:")
print(classification_report(y_test, y_pred_test))


Training Set Evaluation:
              precision    recall  f1-score   support

           0       0.99      0.98      0.99       803
           1       0.67      0.76      0.71        37

    accuracy                           0.97       840
   macro avg       0.83      0.87      0.85       840
weighted avg       0.97      0.97      0.97       840

Test Set Evaluation:
              precision    recall  f1-score   support

           0       0.99      1.00      0.99       197
           1       1.00      0.85      0.92        13

    accuracy                           0.99       210
   macro avg       0.99      0.92      0.96       210
weighted avg       0.99      0.99      0.99       210
